In [10]:
import pandas as pd
import os
import sys
import yaml
import numpy as np

In [15]:
## Mean holdings
Instruments = pd.read_csv('../Average_holdings.csv')
mean_holding_period = round(Instruments.days.mean())
print('mean holding period:', mean_holding_period, 'days')
Inst_dict = dict(zip(Instruments.Symbol, Instruments.days))

mean holding period: 23 days


In [16]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [17]:
original_path = 'ourpped/'
new_dir_path = 'testing_oris/'

from os import listdir
KDD_tickers = [f.split('.')[0] for f in listdir(original_path)]
#KDD_tickers.remove('')
print(KDD_tickers)
print('num of intersection between lists:', len(set(Inst_dict.keys()).intersection(KDD_tickers)))
### create dict for mean holdings for stocknet
KDD_tickers_dict = {}
for s in KDD_tickers:
    try:
        KDD_tickers_dict[s] = int(Inst_dict[s])
    except:
        KDD_tickers_dict[s] = int(mean_holding_period)

['AAPL', 'AMZN', 'BA', 'BAC', 'BHP', 'BRK-B', 'CHL', 'CMCSA', 'CVX', 'D', 'DIS', 'DUK', 'EXC', 'GE', 'HD', 'INTC', 'JNJ', 'JPM', 'KO', 'MA', 'MMM', 'MO', 'MRK', 'MSFT', 'ORCL', 'PEP', 'PFE', 'PG', 'RIO', 'SO', 'SPY', 'T', 'TM', 'UNH', 'UPS', 'VALE', 'VZ', 'WFC', 'WMT', 'XOM']
num of intersection between lists: 34


In [21]:
s='BHP'
stock_movement_path = os.path.join(original_path, '{}.csv'.format(s))  
df = pd.read_csv(stock_movement_path, header=None)
#append mean holdings per stock
mean_holding = KDD_tickers_dict[s]
#save the last to column - will be replaced later
tmp_last_columns = df.iloc[:,-2:].copy()

## add my own features ##
#1) avergae mean per each stock
df.iloc[:,-2] = df.iloc[:,4].rolling(mean_holding).mean()

#2) up most of the days
df.iloc[:,-1] = df.iloc[:,4].rolling(mean_holding).agg(lambda x: (x > 0).mean())

df_new = pd.concat([df, pd.DataFrame(tmp_last_columns)], axis=1)
df_new = df_new.round(6)
df_new.to_csv(os.path.join(new_dir_path,'{}_testing2.csv'.format(s)),header=None, index=None)